### wandb

In [4]:
!pip install wandb

  Using cached wandb-0.13.9-py2.py3-none-any.whl (2.0 MB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached pathtools-0.1.2-py3-none-any.whl
  Using cached setproctitle-1.3.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached sentry_sdk-1.14.0-py2.py3-none-any.whl (178 kB)


In [2]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 1qqqaa. Use `wandb login --relogin` to force relogin


True

sweep

In [10]:
sweep_config = {
    'name' : 'test',
    'method' : 'random',
    'metric' : {
        'name' : 'train/accuracy',
        'goal' : 'maximize'     # 추적할 목표
    },
    'parameters' : {
        # 'optimizer': {
        #     'values' : ['Adam', 'RMSprop']
        # },
        'num_train_triplets' : {
            'values' : [2000, 3000, 4000]
        },
        'num_epochs' : {
            # 'values' : [100, 150]
            'values' : [2,5]
        },
        'batch_size' : {
            'values' : [16, 32]
        },
        # 'step_size' : {
        #     'values' : [50, 100]
        # },
        'learning_rate' : {
            'values' : [0.001, 0.0001]
        }
        
    }
}

In [11]:
sweep_id = wandb.sweep(sweep_config, project="sweep_test") # 프로젝트명

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: wd9tthcg
Sweep URL: https://wandb.ai/1qqqaa/sweep_test/sweeps/wd9tthcg


In [12]:
import pprint

pprint.pprint(sweep_config) # config 확인

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'train/accuracy'},
 'name': 'test',
 'parameters': {'batch_size': {'values': [16, 32]},
                'learning_rate': {'values': [0.001, 0.0001]},
                'num_epochs': {'values': [2, 5]},
                'num_train_triplets': {'values': [2000, 3000, 4000]}}}


train

In [3]:
import argparse
import datetime
import time

import numpy as np
import torch
import torch.optim as optim
from torch.nn.modules.distance import PairwiseDistance
from torch.optim import lr_scheduler

from data_loader import get_dataloader
from datasets.write_csv_for_making_dataset import write_csv
from eval_metrics import evaluate, plot_roc
from loss import TripletLoss
from models import FaceNetModel, model_921
from utils import ModelSaver, init_log_just_created
import wandb
# from inception_resnet_v1 import InceptionResnetV1

parser = argparse.ArgumentParser(description='Face Recognition using Triplet Loss')

parser.add_argument('--num-epochs', default=100, type=int, metavar='NE',
                    help='number of epochs to train (default: 200)')
parser.add_argument('--num-train-triplets', default=4000, type=int, metavar='NTT',
                    help='number of triplets for training (default: 10000)')
parser.add_argument('--num-valid-triplets', default=1000, type=int, metavar='NVT',
                    help='number of triplets for vaidation (default: 10000)')
parser.add_argument('--batch-size', default=16, type=int, metavar='BS',
                    help='batch size (default: 128)')
parser.add_argument('--num-workers', default=8, type=int, metavar='NW',
                    help='number of workers (default: 8)')
parser.add_argument('--learning-rate', default=0.0001, type=float, metavar='LR',
                    help='learning rate (default: 0.001)')
parser.add_argument('--margin', default=0.5, type=float, metavar='MG',
                    help='margin (default: 0.5)')
parser.add_argument('--train-root-dir', default='/home/jupyter/data/face-image/training_aihub_family/', type=str,
                    help='path to train root dir')
parser.add_argument('--valid-root-dir', default='/home/jupyter/data/face-image/training_aihub_family/', type=str,
                    help='path to valid root dir')
parser.add_argument('--train-csv-name', default='datasets/train_dataset.csv', type=str,
                    help='list of training images')
parser.add_argument('--valid-csv-name', default='datasets/val_dataset.csv', type=str, help='list of validtion images')
parser.add_argument('--step-size', default=50, type=int, metavar='SZ',
                    help='Decay learning rate schedules every --step-size (default: 50)')
parser.add_argument('--unfreeze', type=str, metavar='UF', default='',
                    help='Provide an option for unfreezeing given layers')
parser.add_argument('--freeze', type=str, metavar='F', default='',
                    help='Provide an option for freezeing given layers')
parser.add_argument('--pretrain', action='store_true')
parser.add_argument('--fc-only', action='store_true')
parser.add_argument('--except-fc', action='store_true')
parser.add_argument('--load-best', action='store_true')
parser.add_argument('--load-last', action='store_true')
parser.add_argument('--continue-step', action='store_true')
parser.add_argument('--train-all', action='store_true', help='Train all layers')

# args = parser.parse_args()
#주피터에서 사용
args = parser.parse_args(args=[])
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
l2_dist = PairwiseDistance(2)
modelsaver = ModelSaver()


def save_if_best(state, acc):
    modelsaver.save_if_best(acc, state)

# main
def main():
    init_log_just_created("log/log4/valid.csv")
    init_log_just_created("log/log4/train.csv")
    import pandas as pd
    valid = pd.read_csv('log/log4/valid.csv')
    max_acc = valid['acc'].max()

    pretrain = args.pretrain
    fc_only = args.fc_only
    except_fc = args.except_fc
    train_all = args.train_all
    unfreeze = args.unfreeze.split(',')
    freeze = args.freeze.split(',')
    start_epoch = 0
    print(f"Transfer learning: {pretrain}")
    print("Train fc only:", fc_only)
    print("Train except fc:", except_fc)
    print("Train all layers:", train_all)
    print("Unfreeze only:", ', '.join(unfreeze))
    print("Freeze only:", ', '.join(freeze))
    print(f"Max acc: {max_acc:.4f}")
    print(f"Learning rate will decayed every {args.step_size}th epoch")
    
    # model = FaceNetModel()
    model = model_921()
    model.to(device)
    # model = InceptionResnetV1(classify=True, pretrained='vggface2').to(device)
    triplet_loss = TripletLoss(args.margin).to(device)

    wandb.init(
      # Set the project where this run will be logged
      project="face_identification", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{4}", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.0001,
      "architecture": "FaceNet",
      "dataset": "training_aihub",
      "batch_size" : 16,
      "epochs": 100,
      "num-train-triplets" : 4000,
      "num-val-triplets" : 1000,
      "optimizer": "Adam",
      "loss function":"Triplet Loss",
      "step_size" : 50,
      })
    
    # with wandb.init(config=config):
    #     config = wandb.config
        # 이하 필요한 config는 config.size 등으로 사용
        
        # wandb.log({'AUC': max_auc, 'Loss Mean': loss_mean})
        # # 기록해서 추적할 log 설정

    if fc_only:
        model.unfreeze_only(['fc', 'classifier'])
    if except_fc:
        model.freeze_only(['fc', 'classifier'])
    if train_all:
        model.unfreeze_all()
    if len(unfreeze) > 0:
        model.unfreeze_only(unfreeze)
    if len(freeze) > 0:
        model.freeze_only(freeze)

    # if optimizer == 'Adam':
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=args.learning_rate)
    # elif optimizer == 'RMSprop' :
    #     optimizer = optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=args.learning_rate)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=0.1)

    if args.load_best or args.load_last:
        checkpoint = './log/log4/best_state.pth' if args.load_best else './log/log4/last_checkpoint.pth'
        print('loading', checkpoint)
        checkpoint = torch.load(checkpoint)
        modelsaver.current_acc = max_acc
        start_epoch = checkpoint['epoch'] + 1
        model.load_state_dict(checkpoint['state_dict'])
        print("Stepping scheduler")
        try:
            optimizer.load_state_dict(checkpoint['optimizer_state'])
        except ValueError as e:
            print("Can't load last optimizer")
            print(e)
        if args.continue_step:
            scheduler.step(checkpoint['epoch'])
        print(f"Loaded checkpoint epoch: {checkpoint['epoch']}\n"
            f"Loaded checkpoint accuracy: {checkpoint['accuracy']}\n"
            f"Loaded checkpoint loss: {checkpoint['loss']}")

    model = torch.nn.DataParallel(model)

    for epoch in range(start_epoch, args.num_epochs + start_epoch):
        print(80 * '=')
        print('Epoch [{}/{}]'.format(epoch, args.num_epochs + start_epoch - 1))

        time0 = time.time()
        data_loaders, data_size = get_dataloader(args.train_root_dir, args.valid_root_dir,
                                                args.train_csv_name, args.valid_csv_name,
                                                # args.num_train_triplets
                                                args.num_train_triplets, args.num_valid_triplets,
                                                args.batch_size, args.num_workers)

        train_valid(model, optimizer, triplet_loss, scheduler, epoch, data_loaders, data_size)
        print(f'  Execution time                 = {time.time() - time0}')
    print(80 * '=')

def save_last_checkpoint(state):
    torch.save(state, 'log/log4/last_checkpoint.pth')
    # torch.save(state, 'sweep_log/last_checkpoint.pth')
    # torch.save('log1/last_checkpoint.pth')


def train_valid(model, optimizer, triploss, scheduler, epoch, dataloaders, data_size):
    for phase in ['train', 'valid']:

        labels, distances = [], []
        triplet_loss_sum = 0.0

        if phase == 'train':
            scheduler.step()
            if scheduler.last_epoch % scheduler.step_size == 0:
                print("LR decayed to:", ', '.join(map(str, scheduler.get_lr())))
            model.train()
        else:
            model.eval()

        for batch_idx, batch_sample in enumerate(dataloaders[phase]):

            anc_img = batch_sample['anc_img'].to(device)
            pos_img = batch_sample['pos_img'].to(device)
            neg_img = batch_sample['neg_img'].to(device)

            # pos_cls = batch_sample['pos_class'].to(device)
            # neg_cls = batch_sample['neg_class'].to(device)

            with torch.set_grad_enabled(phase == 'train'):

                # anc_embed, pos_embed and neg_embed are encoding(embedding) of image
                anc_embed, pos_embed, neg_embed = model(anc_img), model(pos_img), model(neg_img)

                # choose the semi hard negatives only for "training"
                pos_dist = l2_dist.forward(anc_embed, pos_embed)
                neg_dist = l2_dist.forward(anc_embed, neg_embed)

                all = (neg_dist - pos_dist < args.margin).cpu().numpy().flatten()
                if phase == 'train':
                    hard_triplets = np.where(all == 1)
                    if len(hard_triplets[0]) == 0:
                        continue
                else:
                    hard_triplets = np.where(all >= 0)

                anc_hard_embed = anc_embed[hard_triplets]
                pos_hard_embed = pos_embed[hard_triplets]
                neg_hard_embed = neg_embed[hard_triplets]

                anc_hard_img = anc_img[hard_triplets]
                pos_hard_img = pos_img[hard_triplets]
                neg_hard_img = neg_img[hard_triplets]

                # pos_hard_cls = pos_cls[hard_triplets]
                # neg_hard_cls = neg_cls[hard_triplets]

                model.module.forward_classifier(anc_hard_img)
                model.module.forward_classifier(pos_hard_img)
                model.module.forward_classifier(neg_hard_img)

                triplet_loss = triploss.forward(anc_hard_embed, pos_hard_embed, neg_hard_embed)

                if phase == 'train':
                    optimizer.zero_grad()
                    triplet_loss.backward()
                    optimizer.step()

                distances.append(pos_dist.data.cpu().numpy())
                labels.append(np.ones(pos_dist.size(0)))

                distances.append(neg_dist.data.cpu().numpy())
                labels.append(np.zeros(neg_dist.size(0)))

                triplet_loss_sum += triplet_loss.item()

        avg_triplet_loss = triplet_loss_sum / data_size[phase]
        labels = np.array([sublabel for label in labels for sublabel in label])
        distances = np.array([subdist for dist in distances for subdist in dist])

        tpr, fpr, accuracy, val, val_std, far = evaluate(distances, labels)
        print('  {} set - Triplet Loss       = {:.8f}'.format(phase, avg_triplet_loss))
        print('  {} set - Accuracy           = {:.8f}'.format(phase, np.mean(accuracy)))

        time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        lr = '_'.join(map(str, scheduler.get_lr()))
        layers = '+'.join(args.unfreeze.split(','))
        write_csv(f'log/log4/{phase}.csv', [time, epoch, np.mean(accuracy), avg_triplet_loss, layers, args.batch_size, lr])

        wandb.log({f'{phase}/accuracy':np.mean(accuracy), 
                    f'{phase}/loss':avg_triplet_loss,
                    'learning_rate':lr})

        if phase == 'valid':
            save_last_checkpoint({'epoch': epoch,
                                  'state_dict': model.module.state_dict(),
                                  'optimizer_state': optimizer.state_dict(),
                                  'accuracy': np.mean(accuracy),
                                  'loss': avg_triplet_loss
                                  })
            save_if_best({'epoch': epoch,
                          'state_dict': model.module.state_dict(),
                          'optimizer_state': optimizer.state_dict(),
                          'accuracy': np.mean(accuracy),
                          'loss': avg_triplet_loss
                          }, np.mean(accuracy))
        else:
            if epoch==99:
                plot_roc(fpr, tpr, figure_name='./log/log4/roc_valid_epoch_{}.png'.format(epoch))


In [4]:
main()
wandb.finish()

Transfer learning: False
Train fc only: False
Train except fc: False
Train all layers: False
Unfreeze only: 
Freeze only: 
Max acc: nan
Learning rate will decayed every 50th epoch


/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:136: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  f"Using {sequence_to_str(tuple(keyword_only_kwargs.keys()), separate_last='and ')} as positional "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 1qqqaa. Use `wandb login --relogin` to force relogin


Epoch [0/99]


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  train set - Triplet Loss       = 0.03267236
  train set - Accuracy           = 0.65051227


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:372: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


  valid set - Triplet Loss       = 0.01373763
  valid set - Accuracy           = 0.72550000
  Execution time                 = 217.41949605941772
Epoch [1/99]
  train set - Triplet Loss       = 0.03030698
  train set - Accuracy           = 0.65175639
  valid set - Triplet Loss       = 0.01286995
  valid set - Accuracy           = 0.73500000
  Execution time                 = 207.862548828125
Epoch [2/99]
  train set - Triplet Loss       = 0.02904614
  train set - Accuracy           = 0.63637199
  valid set - Triplet Loss       = 0.01237898
  valid set - Accuracy           = 0.75950000
  Execution time                 = 194.30331778526306
Epoch [3/99]
  train set - Triplet Loss       = 0.02735203
  train set - Accuracy           = 0.66067986
  valid set - Triplet Loss       = 0.01343271
  valid set - Accuracy           = 0.73550000
  Execution time                 = 196.56462025642395
Epoch [4/99]
  train set - Triplet Loss       = 0.03018843
  train set - Accuracy           = 0.6390962

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:372: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


  Execution time                 = 184.29932641983032


train/accuracy,▂▁▁▂▂▂▅▄▃▃▅▃▂▃▄▄▃▅▆▆▅▆▆▇▅█▄▆▅▄▆▅▇▅▅█▇▇█▆
train/loss,█▆▇▆▆▄▄▅▄▄▄▄▄▃▄▅▄▄▃▂▂▁▂▂▂▁▂▂▂▂▂▃▂▂▁▁▂▂▂▂
valid/accuracy,▁▃▁▄▃▃▄▄▅▆▅▅▅▆▄▅▆▅▅▆▅▆▆▆▆▆▆▇▆▇▅▇██▇▇▆▆▆▇
valid/loss,█▇█▅▆▆▆▅▅▅▄▄▄▄▅▅▃▃▄▃▃▃▂▃▃▂▂▂▃▂▃▂▂▁▂▂▂▂▂▂
learning_rate,1.0000000000000002e-...
train/accuracy,0.68812
train/loss,0.01824
valid/accuracy,0.8175
valid/loss,0.00765


In [3]:
main()
wandb.finish()

Transfer learning: False
Train fc only: False
Train except fc: False
Train all layers: False
Unfreeze only: 
Freeze only: 
Max acc: nan
Learning rate will decayed every 50th epoch


/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:136: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  f"Using {sequence_to_str(tuple(keyword_only_kwargs.keys()), separate_last='and ')} as positional "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 1qqqaa. Use `wandb login --relogin` to force relogin


Epoch [0/199]


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


  train set - Triplet Loss       = 0.03457204
  train set - Accuracy           = 0.62630198


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:372: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


  valid set - Triplet Loss       = 0.01501632
  valid set - Accuracy           = 0.70800000


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:372: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


  Execution time                 = 160.60926151275635
Epoch [1/199]
  train set - Triplet Loss       = 0.03270978
  train set - Accuracy           = 0.63802083
  valid set - Triplet Loss       = 0.01368107
  valid set - Accuracy           = 0.72900000
  Execution time                 = 173.95394730567932
Epoch [2/199]
  train set - Triplet Loss       = 0.03095159
  train set - Accuracy           = 0.64730212
  valid set - Triplet Loss       = 0.01403632
  valid set - Accuracy           = 0.72150000
  Execution time                 = 386.26569628715515
Epoch [3/199]
  train set - Triplet Loss       = 0.03298256
  train set - Accuracy           = 0.65382865
  valid set - Triplet Loss       = 0.01541249
  valid set - Accuracy           = 0.70350000
  Execution time                 = 394.8068280220032
Epoch [4/199]
  train set - Triplet Loss       = 0.02667256
  train set - Accuracy           = 0.65338735
  valid set - Triplet Loss       = 0.01537808
  valid set - Accuracy           = 0.68

train/accuracy,▂▃▂▆▅▅▄▅█▂▂▄▅▃▂▄▅▆▁▇▆▁█▇▆▃▅▆▅▄▅▇▃▅▃▄▅▆▅▄
train/loss,█▆▇▅▂▅▅▄▄▄▄▄▃▃▂▄▄▂▃▁▃▃▂▂▂▂▄▂▃▄▂▃▄▃▂▄▂▂▃▁
valid/accuracy,▂▁▂▃▃▄▃▅▅▅▅▅▅▇▇▇▅▆▆▆▇▇▇█▇▇▆▅▇█▆▆▆▇▆▆▆█▇█
valid/loss,▇█▆▅▇▆▆▅▄▄▄▃▄▃▃▂▃▂▃▃▃▃▁▁▂▃▃▄▃▂▃▂▂▂▃▂▂▂▂▁
learning rate,1.0000000000000005e-...
train/accuracy,0.65814
train/loss,0.01546
valid/accuracy,0.817
valid/loss,0.00808


# CUDA out of memory  
먼저 nvidia-smi로 백그라운드에서 실행중인 프로세스가 있는지 확인합니다.

아래 화면에서는 No running processes found라고 뜨는데요, 만약 백그라운드에서 실행중인 프로세스가 있다면 실행중인 프로세스의 목록이 뜨게 됩니다.  
해당 프로세스들이 GPU Memory Usage를 사용하고 있기 때문에, cuda out of memory 에러가 발행하는 것인데요, 만약 현재 실행하려는 프로세스가 아니라면, PID를 이용하여 프로세스를 kill해줍니다. 가끔 서버가 비정상적으로 종료된 경우, 프로세스가 정상적으로 종료되지 않고 백그라운드에 남을 수 있습니다.  

kill -15 [PID]  
만약 이 코드로도 프로세스가 종료되지 않는다면, 아래의 코드를 실행해주세요.

kill -9 [PID]

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [4]:
import torch
torch.cuda.empty_cache()